# Coral Area Generator

---



## Download the this folder with specific formats and ml models into your google drive: https://drive.google.com/drive/folders/1I2PxS79XVj3VTLM-lKzeGh2WqyUNf9IB?usp=sharing

## There is also a readme file to help initial users navigate this notebook more efficiently.

## Install Dependencies and Libraries

---



In [ ]:
!git clone https://github.com/facebookresearch/sam2.git
%cd sam2
!pip install -e . --no-build-isolation

Cloning into 'sam2'...
remote: Enumerating objects: 1070, done.
remote: Total 1070 (delta 0), reused 0 (delta 0), pack-reused 1070 (from 1)
Receiving objects: 100% (1070/1070), 134.70 MiB | 16.62 MiB/s, done.
Resolving deltas: 100% (375/375), done.
/content/sam2
Obtaining file:///content/sam2
  Checking if build backend supports build_editable ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 49.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 32.8 MB/s eta 0:00:00


## Run the Code

In [ ]:
%%writefile Coral_Area_Generator.py
import numpy as np
from PIL import Image
from PIL.ExifTags import TAGS
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import torchvision
import os
import ultralytics
from ultralytics import YOLO
from matplotlib import pyplot as plt
import pandas as pd
import torch
from google.colab import drive, files
import json
import imageio.v3 as iio
import datetime
import cv2
import gc
import torch
from sam2.build_sam import build_sam2
from sam2.sam2_image_predictor import SAM2ImagePredictor
drive.mount('/content/gdrive/')
print(torch.__version__)
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(DEVICE)
young_checkpoint = "/content/gdrive/MyDrive/Coral Area Generator Folder/Machine Learning Models/best_V3_Young_SAM2.pth"
mature_checkpoint = "/content/gdrive/MyDrive/Coral Area Generator Folder/Machine Learning Models/best_V3_Mature_SAM2.pth"
model_cfg = "configs/sam2.1/sam2.1_hiera_l.yaml"
young_predictor = SAM2ImagePredictor(build_sam2(model_cfg, young_checkpoint))
mature_predictor = SAM2ImagePredictor(build_sam2(model_cfg, mature_checkpoint))
def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([30/255, 144/255, 255/255, 0.2]) #0.6
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_points(coords, labels, ax, marker_size=375): #Extra method for the SAM inputs that we don't use
    pos_points = coords[labels==1]
    neg_points = coords[labels==0]
    ax.scatter(pos_points[:, 0], pos_points[:, 1], color='green', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)
    ax.scatter(neg_points[:, 0], neg_points[:, 1], color='red', marker='*', s=marker_size, edgecolor='white', linewidth=1.25)

def show_box(box, ax): #For debugging purposes to check if YOLO is working
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    rect = plt.Rectangle((x0, y0), w, h, edgecolor='green', facecolor=(0,0,0,0), lw=2)
    ax.add_patch(rect)

def show_polygon(polygons, ax):
    formatted_polygons = [np.array(polygon).reshape(-1, 2) for polygon in polygons]
    for polygon in formatted_polygons:
        polygon_patch = patches.Polygon(polygon, closed=True, edgecolor='red', facecolor=(0,0,0,0), lw=2)
        ax.add_patch(polygon_patch)

def show_ground_truth(area, ax):
    ax.text(50, 50, str(area) + " µm", color='white', fontsize=16, backgroundcolor='black')

def get_area(mask): #Prints the mask onto the image in a blue color
    true_count = np.count_nonzero(mask)
    return true_count

def mask_to_polygon(mask):
    # Convert mask to binary if it is not already
    if mask.max() > 1:
        _, binary_mask = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    else:
        binary_mask = (mask * 255).astype(np.uint8)

    # Find contours
    contours, _ = cv2.findContours(binary_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    polygons = []
    for contour in contours:
        # Simplify the contour to reduce the number of points
        epsilon = 0.001 * cv2.arcLength(contour, True)
        approx = cv2.approxPolyDP(contour, epsilon, True)

        # Extract points and flatten the list
        polygon = approx.reshape(-1, 2).tolist()
        flat_polygon = [point for sublist in polygon for point in sublist]
        polygons.append(flat_polygon)

    return polygons

def get_physical_tag(image): #Gets the physical conversion ratios through a library
    physical_size_x = 1
    physical_size_y = 1
    exifdata = image.getexif()
    for tagid in exifdata:
        tagname = TAGS.get(tagid, tagid)
        if tagname == "ImageDescription":
            try:
                value = exifdata.get(tagid)
                start_index_x = value.find('PhysicalSizeX="') + len('PhysicalSizeX="')
                end_index_x = value.find('"', start_index_x)
                physical_size_x = value[start_index_x:end_index_x]
                start_index_x_unit = value.find('PhysicalSizeXUnit="') + len('PhysicalSizeXUnit="')
                end_index_x_unit = value.find('"', start_index_x_unit)
                physical_size_x_unit = value[start_index_x_unit:end_index_x_unit]
                start_index_y = value.find('PhysicalSizeY="') + len('PhysicalSizeY="')
                end_index_y = value.find('"', start_index_y)
                physical_size_y = value[start_index_y:end_index_y]
                start_index_y_unit = value.find('PhysicalSizeYUnit="') + len('PhysicalSizeYUnit="')
                end_index_y_unit = value.find('"', start_index_y_unit)
                physical_size_y_unit = value[start_index_y_unit:end_index_y_unit]
            except Exception as e:
                print(f'Failed to process {image}: {e}')
                continue
    return float(physical_size_x), float(physical_size_y)

def get_physical(image_path):
    physical_size_x = 1
    physical_size_y = 1
    try:
        # Retrieve metadata
        metadata = iio.immeta(image_path)

        # Debugging output to understand metadata structure
        print(f"Metadata for {image_path}: {metadata}")

        # Ensure the 'pixelsizex' and 'pixelsizey' keys are present in the metadata
        physical_size_x = metadata.get('pixelsizex', 'N/A')
        physical_size_y = metadata.get('pixelsizey', 'N/A')

        # Check if the physical sizes are retrieved correctly
        if physical_size_x == 'N/A' or physical_size_y == 'N/A':
            physical_size_x = 1
            physical_size_y = 1
        else :
            physical_size_x = float(physical_size_x) * 10**6
            physical_size_y = float(physical_size_y) * 10**6

    except Exception as e:
        print(f'Failed to retrieve metadata from {image_path}: {e}')

    return physical_size_x, physical_size_y

def create_coco_json(images_data, annotations, categories, output_file): #Creates a COCO.json file that allows user to edit annotations else where
    coco_json = {
        "info": {
            "description": "Coral Areas",
            "url": "https://drive.google.com/drive/folders/1I2PxS79XVj3VTLM-lKzeGh2WqyUNf9IB?usp=sharing",
            "version": "1.0",
            "year": 2024,
            "contributor": "Richard Zhao, Eric Su, Simon Zhao, Tracy Chen",
            "date_created": "2024-07-25"
        },
        "licenses": [
            {
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License",
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/"
            }
        ],
        "images": [],
        "annotations": [],
        "categories": categories
    }

    for image in images_data:
        image_info = {
            "id": image["id"],
            "license": image["license"],
            "file_name": image["file_name"],
            "width": image["width"],
            "height": image["height"],
            "date_captured": image["date_captured"]
        }
        coco_json["images"].append(image_info)

    for annotation in annotations:
        annotation_info = {
            "id": annotation["id"],
            "image_id": annotation["image_id"],
            "category_id": annotation["category_id"],
            "bbox": annotation["bbox"],
            "area": annotation["area"],
            "segmentation": annotation["segmentation"],
            "iscrowd": annotation["iscrowd"]
        }
        coco_json["annotations"].append(annotation_info)

    with open(output_file, 'w') as f:
        json.dump(coco_json, f, indent=4)

def execute_block1(image_path): #Opens the image and returns the bounding box
    image = Image.open(image_path)
    image_np = np.array(image)
    height, width, channels = image_np.shape
    model = YOLO("/content/gdrive/MyDrive/Coral Area Generator Folder/Machine Learning Models/best.pt")
    results = model.predict(image, verbose=False)
    classes = ['late-recruits']
    for result in results:
        boxes = result.boxes
    if boxes.xyxy.tolist():
        bbox = boxes.xyxy.tolist()[0]
        class_ids = boxes.cls.tolist()
        classes = [model.names[int(cls_id)] for cls_id in class_ids]
    else:
        bbox = [0, width, 0, height]
    return image, bbox, classes

def execute_block2(image, bbox, classes): #Gets the mask using the image and the bounding box
    input_box = np.array(bbox)
    center = [(bbox[0] + bbox[2]) / 2, (bbox[1] + bbox[3]) / 2]
    top_mid = [(bbox[0] + bbox[2]) / 2, bbox[1]]
    bottom_mid =[(bbox[0] + bbox[2]) / 2, bbox[3]]
    left_mid = [bbox[0], (bbox[1] + bbox[3]) / 2]
    right_mid = [bbox[2], (bbox[1] + bbox[3]) / 2]
    if len(classes) > 0 and classes[0] == 'early-recruits':
        input_point = np.array([center, top_mid, bottom_mid, left_mid, right_mid])
        input_label = np.array([1, 0, 0, 0, 0])
        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            young_predictor.set_image(np.array(image))
            masks, _, _ = young_predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                box=input_box[None, :],
                multimask_output=False,
            )
    else:
        input_point = np.array([center])
        input_label = np.array([1])
        with torch.inference_mode(), torch.autocast("cuda", dtype=torch.bfloat16):
            mature_predictor.set_image(np.array(image))
            masks, _, _ = mature_predictor.predict(
                point_coords=input_point,
                point_labels=input_label,
                box=input_box[None, :],
                multimask_output=False,
            )
    return masks

def execute_block3(image, bbox, masks, annotated_path, index, image_path): #Saves the images into the defined path, and gets the amount of pixels in the mask
    pixels = get_area(masks[0])
    pixel_size_x, pixel_size_y = get_physical(image_path)
    if (pixel_size_x == 1):
        pixel_size_x, pixel_size_y = get_physical_tag(image)
    area = pixel_size_x * pixel_size_y * pixels
    segmentation = mask_to_polygon(masks[0])
    image_file = os.path.basename(annotated_path)
    input_box = np.array(bbox)
    image_np = np.array(image)
    height, width, channels = image_np.shape
    scale_factor = 100
    fig, ax = plt.subplots(figsize=(width / scale_factor, height / scale_factor))
    ax.set_position([0, 0, 1, 1])
    ax.set_axis_off()
    ax.imshow(image, aspect='auto')
    if (vmask == "y"):
        show_mask(masks[0], ax)
    if (vbbox == "y"):
        show_box(input_box, ax)
    if (vpolygon == "y"):
        show_polygon(segmentation, ax)
    if (vground_truth == "y"):
        show_ground_truth(area, ax)
    fig.savefig(annotated_path, dpi=100)
    plt.close(fig)

    image_info = {
        "id": index,
        "license": 1,
        "file_name": image_file,
        "height": height,
        "width": width,
        "date_captured": datetime.datetime.now().isoformat()
    }
    annotation_info = {
        "id": index,
        "image_id": index,
        "category_id": 1,
        "bbox": bbox,
        "area": pixels,
        "segmentation": segmentation,
        "iscrowd": 0
    }

    return pixels, image_info, annotation_info, area, pixel_size_x, pixel_size_y

vmask = input("Show blue mask? (y/n): ")
vbbox = input("Show bounding box? (y/n): ")
vpolygon = input("Show polygon outline? (y/n): ")
vground_truth = input("Show ground truth? (y/n): ")
folder_read_paths = input("Enter file path(s) where your images are located separated by commas (Ex: /content/gdrive/MyDrive/Coral Area Generator Folder/Coral Recruit Images/, ...): ")
folder_read_paths = [path.strip() for path in folder_read_paths.split(",")]
folder_write_path = input("Enter file path where you want to output images (Ex: /content/gdrive/MyDrive/Coral Area Generator Folder/Code Output/): ")
csv_path = folder_write_path + "coral_areas_output.csv"
columns = ["Folder", "Image Name", "Class", "Pixel Area", "Pixel Size X", "Pixel Size Y", "µm^2"]
unique_folder_names = {}

for index, folder in enumerate(folder_read_paths):

    images_data = []
    annotations = []
    categories = [
        {"id":0,"name":"coral","supercategory":"none"},
        {"id":1,"name":"coral","supercategory":"coral"}
    ]
    folder_name = os.path.basename(folder.rstrip('/'))

    if folder_name in unique_folder_names:
        unique_folder_names[folder_name] += 1
        new_folder_name = f"{folder_name}_{unique_folder_names[folder_name]}"
    else:
        unique_folder_names[folder_name] = 0
        new_folder_name = folder_name

    new_folder_path = os.path.join(folder_write_path, folder_name)
    if not os.path.exists(new_folder_path):
        os.makedirs(new_folder_path)

    json_path = os.path.join(new_folder_path, "_annotations.coco.json")

    image_index = 0
    for image_file in os.listdir(folder):
        if image_file.endswith('.tif'):
            image_path = os.path.join(folder, image_file)
            base_name, _ = os.path.splitext(image_file)
            image_file_png = base_name + ".png"
            annotated_path = os.path.join(new_folder_path, image_file_png)
            image, bbox, classes = execute_block1(image_path)
            masks = execute_block2(image, bbox, classes)
            pixels, image_info, annotation_info, area, pixel_size_x, pixel_size_y= execute_block3(image, bbox, masks, annotated_path, image_index, image_path)
            images_data.append(image_info)
            annotations.append(annotation_info)
            new_row = {
                "Folder": new_folder_name,
                "Image Name": image_file,
                "Class": classes[0] if classes else "No Class Detected",  # Default to a string if classes is empty
                "Pixel Area": pixels,
                "Pixel Size X": pixel_size_x,
                "Pixel Size Y": pixel_size_y,
                "µm^2": area
            }
            new_row_df = pd.DataFrame([new_row], columns=columns)
            new_row_df.to_csv(csv_path, mode='a', header=not os.path.exists(csv_path), index=False)
            print(
                "Folder: " + new_folder_name + "\n" +
                "Image Name: " + image_file + "\n" +
                "Class: " + (str(classes[0]) if classes else "No Class Detected") + "\n" +
                "Bounding Box: " + str(bbox) + "\n" +
                "Pixels: " + str(pixels) + "\n" +
                "Pixel Size X: " + str(pixel_size_x) + "\n" +
                "Pixel Size Y: " + str(pixel_size_y) + "\n" +
                "Area: " + str(area) + " µm^2" + "\n"
            )
            image_index += 1
            gc.collect()
            torch.cuda.empty_cache()

    create_coco_json(images_data, annotations, categories, json_path)

Writing Coral_Area_Generator.py


In [ ]:
%run Coral_Area_Generator.py

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Mounted at /content/gdrive/
2.6.0+cu124
cuda:0
Show blue mask? (y/n): n
Show bounding box? (y/n): y
Show polygon outline? (y/n): y
Show ground truth? (y/n): y
Enter file path(s) where your images are located separated by commas (Ex: /content/gdrive/MyDrive/Coral Area Generator Folder/Coral Recruit Images/, ...): /content/gdrive/MyDrive/Coral Area Generator Folder/Coral Recruit Images/,
Enter file path where you want to output images (Ex: /content/gdrive/MyDrive/Coral Area Generator Folder/Code Output/): /content/gdrive/MyDrive/Coral Area Generator Folder/Code Output/
Failed to retrieve metadata from /content/gdrive/MyDrive/Coral Area Generator Folder/Coral Recruit Images/108_2_202

FileNotFoundError: [Errno 2] No such file or directory: ''

<Figure size 640x480 with 0 Axes>